In [1]:
from __future__ import absolute_import, division
from math import log

def eflux(model, expression_profile, condition_header="condition", control_header="control"):
    """
    E-Flux[1]
    
    Parameters
    ----------
    model: cobra.Model
        The constraint-based model to perform E=Flux on.
    expression_profile: ExpressionProfile
        An expression profile to integrate in the model.
    condition_header: str or int, optional (default "condition")
        The header of the condition column in the expression profile.
    control_header: str or int, optional (default "control")
        The header of the control column in the expression profile.
    
    Returns
    -------
    context-specific model: cobra.Model
    log(condition_sol/control_sol): float
    
    References
    ----------
    .. [1] Colijn C, Brandes A, Zucker J, Lun DS, Weiner B, et al. (2009)
           Interpreting Expression Data with Metabolic Flux Models:
           Predicting Mycobacterium tuberculosis Mycolic Acid Production.
           PLoS Comput Biol 5(8): e1000489.
           doi:10.1371/journal.pcbi.1000489
    """
    
    assert isinstance(model, cobra.Model)
    assert isinstance(expression_profile, ExpressionProfile)
    assert isinstance(condition_header, (str, int))
    assert isinstance(control_header, (str, int))
    
    control_exp = expression_profile.to_reaction_dict(control_header, model,
                                                      normalize_by="eflux")
    condition_exp = expression_profile.to_reaction_dict(condition_header, model,
                                                        normalize_by="eflux")
    _, control_max = expression_profile.minmax(condition=control_header)
    _, condition_max = expression_profile.minmax(condition=condition_header)
    
    with model:
        # Exchange reaction bound alteration
        for ex_rxn in model.exchanges:
            ex_rxn.bounds = (-1.0, 1.0)
        # Control reaction bound alteration
        for rxn_id, expression in six.iteritems(control_exp):
            rxn = model.reactions.get_by_id(rxn_id)
            rxn.upper_bound = expression / control_max
        control_sol = model.optimize()
        # Condition reaction bound alteration
        for rxn_id, expression in six.iteritems(condition_exp):
            rxn = model.reactions.get_by_id(rxn_id)
            rxn.upper_bound = expression / condition_max
        condition_sol = model.optimize()   
        return (condition_sol, log(condition_sol.objective_value/control_sol.objective_value))